In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def make_get_request(url, params=None):
    """
    Функция для выполнения GET-запроса.

    :param url: URL для выполнения запроса.
    :param params: Словарь с параметрами запроса (по умолчанию None).
    :return: Объект response или None в случае ошибки.
    """
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Проверка на ошибки HTTP
        print('Visited URL: {}'.format(response.url))
        print(response.status_code)
        return response
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при выполнении запроса: {e}")
        return None

def extract_content(soup):
    """
    Извлекает только оглавление из объекта BeautifulSoup.

    :param soup: Объект BeautifulSoup.
    :return: Список строк оглавления.
    """
    try:
        # Находим главный div
        content_div = soup.find('div', {'id': 'content', 'class': 'tab-pane active'})

        if not content_div:
            print("Не найден div с id='content' и class='tab-pane active'")
            """return []"""

        # Получаем все текстовые элементы внутри div
        text_elements = content_div.find_all(string=True, recursive=True)

        # Фильтруем только строки, которые похожи на пункты оглавления
        content = [element.strip() for element in text_elements if isinstance(element, str) and element.strip() and (element.strip()[0].isdigit() or "ВВЕДЕНИЕ" in element.strip() or "ЗАКЛЮЧЕНИЕ" in element.strip() or "СПИСОК ИСПОЛЬЗОВАННЫХ ИСТОЧНИКОВ" in element.strip() or "ПРИЛОЖЕНИЕ" in element.strip())]

        return content

    except Exception as e:  # Ловим более общие исключения
        print(f"Ошибка при извлечении содержания: {e}")
        return []
        
def extract_subject(soup):
    """
    Извлекает предмет из объекта BeautifulSoup.

    :param soup: Объект BeautifulSoup.
    :return: Строка с названием предмета или None, если не найден.
    """
    try:
        # Находим div с классом "muted"
        muted_div = soup.find('div', class_='muted')

        if not muted_div:
            print("Не найден div с class='muted'")
            return None

        # Находим все li внутри ul внутри muted_div
        li_elements = muted_div.find_all('li')

        # Ищем li, который содержит "Предмет:"
        subject_li = None
        for li in li_elements:
            if "Предмет:" in li.get_text():
                subject_li = li
                break  # Нашли нужный li, выходим из цикла

        if not subject_li:
            print("Не найден li с 'Предмет:'")
            return None

        # Находим span с классом "value" внутри subject_li
        subject_span = subject_li.find('span', class_='value')

        if not subject_span:
            print("Не найден span с class='value'")
            return None

        # Извлекаем текст из span
        subject = subject_span.text.strip()
        return subject

    except Exception as e:
        print(f"Ошибка при извлечении предмета: {e}")
        return None

# Список URL
urls = [
    "https://a-center.ru/gotovye-raboty/internet-prilozhenie-tsvetochnyy-magazin-1901338/",
    "https://a-center.ru/gotovye-raboty/osnovnye-printsipy-modulnogo-programmirovaniya-9300561/",
    "https://a-center.ru/gotovye-raboty/razrabotka-programmnogo-prilozheniya-pokupka-avtomobilya-v-avtosalone-v-srede-obektno-orientirovanno-9300555/",
    "https://a-center.ru/gotovye-raboty/osnovy-raboty-v-integrirovannoy-srede-razrabotki-delphi-osnovnye-komponenty-i-ikh-svoystva-9300583/",
    "https://a-center.ru/gotovye-raboty/proekt-na-yazyke-programmirovaniya-visual-basic-dlya-provedeniya-testirovaniya-1101822/",
    "https://a-center.ru/gotovye-raboty/razrabotka-informatsionnoy-sistemy-dlya-apteki-9100954/"
]

# Параметры
params = {
    'param1': 'value1',
    'param2': 'value2',
    'param3': 'value3',
    'param4': 'value4',
    'param5': 'value5',
    'param6': 'value6'
}

# Списки для хранения результатов
all_headers = []
all_data = [] # Список для хранения словарей с данными

# Обработка каждого URL
for url in urls:
    response = make_get_request(url, params)

    if response:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        # Извлечение заголовков h1
        data_containers = soup.find_all('h1')
        if data_containers:
            headers_for_page = [dc.text for dc in data_containers]
            all_headers.extend(headers_for_page)
        else:
            print(f"Нет заголовков h1 на странице {url}")
            headers_for_page = []

        # Извлечение содержания
        content = extract_content(soup)

        # Извлечение предмета
        subject = extract_subject(soup)

        # Сохраняем данные для URL
        all_data.append({"url": url, "subject": subject, "content": content, "headers": headers_for_page})

    else:
        print(f"Ошибка при запросе к {url}")

# Вывод результатов
print(f"\nСписок всех заголовков h1: {all_headers}")
print("\nДанные для каждого URL:")
for item in all_data:
    print(f"\nURL: {item['url']}")
    print(f"Тема: {item['headers']}")
    print(f"Предмет: {item['subject']}")
    print(f"Содержание: {item['content']}")

Visited URL: https://a-center.ru/gotovye-raboty/internet-prilozhenie-tsvetochnyy-magazin-1901338/?param1=value1&param2=value2&param3=value3&param4=value4&param5=value5&param6=value6
200
Visited URL: https://a-center.ru/gotovye-raboty/osnovnye-printsipy-modulnogo-programmirovaniya-9300561/?param1=value1&param2=value2&param3=value3&param4=value4&param5=value5&param6=value6
200
Visited URL: https://a-center.ru/gotovye-raboty/razrabotka-programmnogo-prilozheniya-pokupka-avtomobilya-v-avtosalone-v-srede-obektno-orientirovanno-9300555/?param1=value1&param2=value2&param3=value3&param4=value4&param5=value5&param6=value6
200
Visited URL: https://a-center.ru/gotovye-raboty/osnovy-raboty-v-integrirovannoy-srede-razrabotki-delphi-osnovnye-komponenty-i-ikh-svoystva-9300583/?param1=value1&param2=value2&param3=value3&param4=value4&param5=value5&param6=value6
200
Visited URL: https://a-center.ru/gotovye-raboty/proekt-na-yazyke-programmirovaniya-visual-basic-dlya-provedeniya-testirovaniya-1101822/?para